In [ ]:
/home/wanted-1/potenup-workspace/Project/project3/team2/SY/Gemma/processed_text.txt

In [1]:
import torch
import os
import re
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel, prepare_model_for_kbit_training

/home/wanted-1/miniconda3/envs/whisper/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda:0


In [3]:
# 1. 설정: 베이스 모델 및 quantization 설정
BASE_MODEL = "google/gemma-2b-it"

#동일한 환경에서 모델과 LoRA 어댑터를 올바르게 불러오기 위해 필요
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

# 2. 베이스 모델 불러오기 및 k-bit 학습 준비
base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    quantization_config=bnb_config,
    device_map={"": torch.cuda.current_device()}
)
base_model = prepare_model_for_kbit_training(base_model)

# 3. 저장한 LoRA 어댑터 불러오기
model = PeftModel.from_pretrained(base_model, "path/to/save/lora_adapters_ver1_law")
model.eval()  # 평가 모드로 전환

Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.79it/s]


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): GemmaForCausalLM(
      (model): GemmaModel(
        (embed_tokens): Embedding(256000, 2048, padding_idx=0)
        (layers): ModuleList(
          (0-17): 18 x GemmaDecoderLayer(
            (self_attn): GemmaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=2048, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
             

In [4]:
# 4. 토크나이저 초기화
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

In [5]:
# ---------------------------------------------------------------------
# 1. 생성 하이퍼파라미터 조정
generation_kwargs = {
    "max_length": 4096,           # 최대 생성 토큰 수 (요약 길이 제한)
    "min_length": 512,            # 최소 생성 토큰 수
    "do_sample": True,           # 샘플링 방식 사용
    "temperature": 0.5,          # 낮은 값은 결정론적, 높은 값은 다양성 증가
    "top_k": 50,                 # 상위 k개 단어 내에서 샘플링
    "top_p": 0.95,               # 누적 확률 p 내 단어에서 샘플링
    "num_return_sequences": 1   # 3개의 후보 요약 생성 (후보 비교를 위해)
}

In [6]:
# 5. 요약할 텍스트 파일 불러오기
txt_file_path = "/home/wanted-1/potenup-workspace/Project/project3/team2/SY/Gemma/processed_text.txt"  # 파일 경로를 수정하세요.
with open(txt_file_path, "r", encoding="utf-8") as f:
    original_text = f.read()


make summary
#  Write the summary in complete sentences, ensuring that each sentence ends with a period.
# - The summary must be composed of at least 3 sentences and no more than 5 sentences.
# - Avoid unnecessary elaboration or repetition; be concise and clear.
# - Accurately reflect the core information of the original text.
# - "했으며" 라는 표현 줄이고, 문장 단위로 작성
# - Avoid using ','. 
# - Accurately reflect the core information of the original text.
# - Do not use compound sentences with connectors like 'and' (minimize expressions similar to "며,"); write each sentence independently.
# - Do not use commas.
# - Make sure the summary is clear, concise, and written in separate sentences.
# - Only output the summary.
# - Do not include any notes, disclaimers, or additional comments.

# After reading the original text below, please write a summary following these guidelines:
# - Summarize in English first
# - Make sure the summary is clear, concise, and written in separate sentences.
# - The summary must be composed of at least 3 sentences and no more than 5 sentences.

# - then translate it into Korean.
# - Translate sentence by sentence, keeping each sentence distinct.
# - Ensure that each translated sentence remains separate and does not merge.
# - Do not write '**Note:**' AND 'Note:'
# - Do not include any notes, disclaimers, or additional comments. Only output the summary.

# - Avoid commas. Write short and clear sentences instead.
# - Provide the summary as bullet points
# - Each bullet point should include the main ideas, such as key events, people, dates, and any significant background.
# - Include all the main ideas and key details of the original text.
# - Do not use ','

#  Write the summary in complete sentences, ensuring that each sentence ends with a period.
# - The summary must be composed of at least 3 sentences and no more than 5 sentences.
# - Avoid unnecessary elaboration or repetition; be concise and clear.
# - Accurately reflect the core information of the original text.
# - "했으며" 라는 표현 줄이고, 문장 단위로 작성
# - Avoid using ','. 
# - Accurately reflect the core information of the original text.
# - Do not use compound sentences with connectors like 'and' (minimize expressions similar to "며,"); write each sentence independently.
# - Do not use commas.
# - Make sure the summary is clear, concise, and written in separate sentences.
# - Only output the summary.
# - Do not include any notes, disclaimers, or additional comments.


In [ ]:
# 3. 상세 프롬프트 구성
# 프롬프트에 구체적인 요약 작성 지시와 예시를 포함하여 모델이 어떤 정보를 포함해야 하는지 명확히 유도합니다.
detailed_prompt = """

After reading the original text below, please perform the following steps in order:

1. Translate the original text into English.
2. Summarize the translated text by extracting only the most important core points. Present each key point as a numbered item (for example, "1.", "2.", "3.", etc.). Write each point concisely and clearly.
3. Finally, translate the numbered summary into Korean.

Important:
- The final output must be only the summary in Korean.
- Use simple and direct language.
- Do not include any extra commentary, notes, or repeated information.
- Each numbered point must be on a separate line and end with a period.
- Do not include any bullet symbols (such as "*" or "-").
- Do not write '**Note:**' AND 'Note:'
- Do not include any notes, disclaimers, or additional comments. Only output the summary.

Original Text:
{original_text}

Summary:
"""

# 실제 원문 텍스트 (예시로 직접 입력하거나 파일에서 읽어올 수 있음)
#original_text = "여기에 실제 원문 텍스트를 입력합니다. 이 텍스트는 예시로서 모델이 요약할 내용입니다."

# 프롬프트 구성
prompt = detailed_prompt.format(original_text=original_text)

# 4. 입력 토큰화
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

In [29]:
# 5. 모델을 통한 요약 생성
with torch.no_grad():
    output_tokens = model.generate(**inputs, **generation_kwargs)

In [30]:
# ---------------------------------------------------------------------
# 4. 후처리 함수 정의: 생성된 텍스트를 정제합니다.
def post_process(text):
    # 앞뒤 공백 제거
    text = text.strip()
    # 불필요한 공백, 줄바꿈, 중복 문장 부호 제거 (예시: 느낌표, 공백 등)
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'!{2,}', '!', text)
    return text


In [31]:
generated_text = tokenizer.decode(output_tokens[0], skip_special_tokens=True)
# "요약:" 이후의 텍스트가 실제 요약이라고 가정
import re

def refine_sentences(text):
    """
    1. 문장을 적절히 분리하고 중복 제거
    2. 잘못 추가된 마침표를 정리하여 자연스럽게 변환
    """
    # 1️⃣ 문장 단위로 나누기 (마침표, 물음표, 느낌표 기준)
    sentences = re.split(r'(?<=[.!?])\s+', text.strip())

    # 2️⃣ 중복 문장 제거
    unique_sentences = []
    seen = set()

    for sentence in sentences:
        cleaned_sentence = sentence.strip()
        if cleaned_sentence and cleaned_sentence not in seen:
            unique_sentences.append(cleaned_sentence)
            seen.add(cleaned_sentence)

    # 3️⃣ 각 문장을 개행 문자("\n")로 연결하여 출력
    return "\n".join(unique_sentences).strip()


# 🔹 생성된 텍스트에서 "Summary:" 이후의 부분만 추출
if "Summary:" in generated_text:
    summary_part = generated_text.split("Summary:")[-1].strip()
else:
    summary_part = generated_text.strip()

# 🔹 중복 제거 후 정리된 문장 적용
final_summary = refine_sentences(summary_part)

print("Final Refined Summary:")
print(final_summary)



Final Refined Summary:
국정원이 북한 해커의 위장 취업을 조심하라 주의령을 내렸는데, 이는 국정원이 국내 한 에너지 기업의 해외지사에 취업하려던 북한 해커를 적발해 조사 중이라고 밝혔으며, 이 해커는 굳이 플랫폼 사이트에 등록하는 대담함도 보였으며, 국정원은 대남 사이버 테러를 주도했던 김영철의 복기도 주시하고 있으며, 채널A 뉴스 배두원이라고 한다.
Note:
국정원은 우리나라가 의제를 선호하듯 북한은 IT 분야 진출을 선호한다고도 했습니다.
국정원은 대남 사이버 테러를 주도했던 김영철의 복기도 주시하고 있다.


In [ ]:

    
# # 8. 생성된 토큰 디코딩 후 출력
# generated_text = tokenizer.decode(generated_tokens[0], skip_special_tokens=True)
# print("생성된 요약:")
# print(generated_text)
